
# 自動機械学習 Automated Machine Learning による自動車価格予測モデリング & モデル解釈 (リモート高速実行)


中古車の属性データから価格を予測するモデルを構築します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 計算環境の準備
- 自動機械学習の事前設定
- モデル学習と結果の確認
- モデル解釈

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-regression-automobile-remote'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning service の Dataset のフォーマットでデータを定義します。

In [5]:
dataset = Dataset.get_by_name(ws, name='automobile')
dataset.take(5).to_pandas_dataframe()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,nan,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
1,3,nan,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
2,1,nan,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,...,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
3,2,164.00,audi,gas,std,four,sedan,fwd,front,99.80,...,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
4,2,164.00,audi,gas,std,four,sedan,4wd,front,99.40,...,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450


In [6]:
train_dataset, test_dataset = dataset.random_split(0.8, seed=1234)

In [7]:
X_train = train_dataset.drop_columns(columns=['price'])
y_train = train_dataset.keep_columns(columns=['price'], validate=True)
X_test  = test_dataset.drop_columns(columns=['price'])
y_test  = test_dataset.keep_columns(columns=['price'], validate=True)

### 計算環境 (Machine Learning Compute) の設定

In [8]:
# 予め cpucluster という名称の Machine Learning Compute を作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(ws, "cpucluster")

In [10]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True

cd = CondaDependencies.create(conda_packages=['numpy','py-xgboost<=0.80', 'tensorflow>=1.10.0,<=1.12.0'])
conda_run_config.environment.python.conda_dependencies = cd

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [11]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 50,
    "n_cross_validations": 3,
    "primary_metric": 'normalized_mean_absolute_error',
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'regression',
                             X = X_train,
                             y = y_train,
                             run_configuration=conda_run_config,
                             max_concurrent_iterations = 10,
                             **automl_settings
                            )

### 実行と結果確認

In [12]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote compute: cpucluster
Parent Run ID: AutoML_1dd2ba9f-38b4-4895-b86c-3eb6506acbd7
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         4   StandardScalerWrapper DecisionTree             0:03:20       0.0571    0.0571
         6   StandardScalerWrapper ElasticNet               0:04:04       0.0498    0.0498
         5   MaxAbsScaler DecisionTree                      0:04:20       0.0499    0.0498
         0   Standa

In [13]:
# Widget で結果確認
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

In [14]:
# 詳細ログの出力
remote_run.get_details()

{'runId': 'AutoML_1dd2ba9f-38b4-4895-b86c-3eb6506acbd7',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-10-02T16:09:40.682395Z',
 'endTimeUtc': '2019-10-02T16:33:43.451841Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'MaxTimeSeconds': '300',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpucluster',
  'RawAMLSettingsString': "{'name': 'automl-regression-automobile-remote', 'path': None, 'subscription_id': '9c0f91b8-eb2f-484c-979c-15848c098a6b', 'resource_group': 'mlservice', 'workspace_name': 'azureml', 'region': 'eastus', 'compute_target': 'cpucluster', 'spark_service': None, 'iterations': 50, 'primary_metric': 'normalized_mean_absolute_error', 'task_type': 'regression', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 3, 'y_min': None, 'y_max': None, 'num_classes': None, 'p

In [15]:
best_run, fitted_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-regression-automobile-remote,AutoML_1dd2ba9f-38b4-4895-b86c-3eb6506acbd7_45,azureml.scriptrun,Completed,Link to Azure Portal,Link to Documentation


### モデルの理解

In [16]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'symboling',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredFeatureCount': 6,
  'Tranformations': ['StringCast-CharGramCountVectorizer']},
 {'RawFeatureName': 'fuel-type',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['ModeCatImputer-StringCast-LabelEncoder']},
 {'RawFeatureName': 'aspiration',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['ModeCatImputer-StringCast-LabelEncoder']},
 {'RawFeatureName': 'num-of-doors',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredFeatureCount': 3,
  'Tranformations': ['StringCast-CharGramCountVectorizer']},
 {'RawFeatureName': 'body-style',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredFeatureCount': 5,
  'Tranformations': ['StringCast-CharGramCountVectorizer']},
 {'RawFeatureName': 'drive-wheels',
  'TypeDetected': 'Categorical',
  'Dropped': 'No',
  'EngineeredF

In [17]:
from pprint import pprint


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()


print_model(fitted_model)

datatransformer
{'enable_feature_sweeping': None,
 'feature_sweeping_timeout': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None}

MaxAbsScaler
{'copy': True}

RandomForestRegressor
{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.001953125,
 'min_samples_split': 0.012814223889440833,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 600,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}



## 3. モデル解釈

In [18]:
from azureml.train.automl.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations

automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, X=X_train, X_test=X_test, y=y_train, task='regression')

Current status: Setting up data for AutoMl explanations
Current status: Setting up the AutoML featurization for explanations
Current status: Setting up the AutoML estimator
Current status: Setting up the AutoML featurizer
Current status: Generating a feature map for raw feature importance
Current status: Data for AutoMl explanations successfully setup


In [19]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

Using older than supported version of lightgbm, please upgrade to version greater than 2.2.1


In [20]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)
#print(raw_explanations.get_feature_importance_dict())

In [21]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
ExplanationDashboard(raw_explanations, automl_explainer_setup_obj.automl_pipeline, automl_explainer_setup_obj.X_test_raw)

ExplanationWidget(value={'predictedY': [7898.397194444443, 6508.518896825397, 11400.780523809519, 34161.025202…